# 4. Stitch outputs together
- Once you have fitted the weak, medium and dense diads, stitch them all together, and apply the Ne correction model

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import joblib
import os
from pickle import load
import pickle
import DiadFit as pf
pf.__version__

'0.0.59'

In [2]:
MasterFolder = r"P:\WORK-GENERAL\POSTDOC-UCB\BERKELEY-VIBE\Documents\Projects\Data\Hawaii_FI\Data\Raman\Leilani-2018"
DayFolder = os.path.join(MasterFolder, "Leilani_2018FI_Ap623")

meta_path=DayFolder +  '\Metadata'
spectra_path=DayFolder + '\Spectra'
filetype='headless_txt'

if not os.path.exists(MasterFolder + '\OUTPUT'):
    os.mkdir(MasterFolder +'\OUTPUT')


output_path=MasterFolder +'\OUTPUT'


## Load in specra, and combine

In [3]:
from os import path
if path.exists('Discarded_df.xlsx'):
    discard=pd.read_excel('Discarded_df.xlsx')
else:
    discard=None
if path.exists('Weak_Diads.xlsx'):
    grp1=pd.read_excel('Weak_Diads.xlsx')
else:
    grp1=None
if path.exists('Medium_Diads.xlsx'):
    grp2=pd.read_excel('Medium_Diads.xlsx')
else:
    grp2=None
if path.exists('Strong_Diads.xlsx'):
    grp3=pd.read_excel('Strong_Diads.xlsx')
else:
    grp3=None
df2=pd.concat([grp1, grp2, grp3], axis=0)
if discard is not None:
    discard_cols=discard[discard.columns.intersection(df2.columns)]
    df2=pd.concat([df2, discard_cols])

In [4]:
df2.head()

,Unnamed: 0,filename,Splitting,Split_err_abs,Split_err_quadrature,Diad1_Combofit_Cent,Diad1_cent_err,Diad1_Combofit_Height,Diad1_Voigt_Cent,Diad1_Voigt_Area,...,HB2_Sigma,C13_Cent,C13_Area,C13_Sigma,Diad2_Gauss_Cent,Diad2_Gauss_Area,Diad2_Gauss_Sigma,Diad1_Gauss_Cent,Diad1_Gauss_Area,Diad1_Gauss_Sigma
0,0,LL1_275-FI#2-R2_CRR_DiadFit,103.294890,0.133473,0.096374,1285.690490,0.080530,49.014499,1285.690490,50.973451,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,0,LL1_275-FI#2-R3_CRR_DiadFit,103.188372,0.037022,0.026692,1285.834584,0.022195,128.202572,1285.834584,144.833070,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,0,LL1_275-FI#2-R4_CRR_DiadFit,103.204924,0.009848,0.009848,1285.782739,0.000000,146.530132,1285.782739,182.468185,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,0,LL1_275-FI#3-R1_CRR_DiadFit,103.138565,0.058238,0.042143,1286.072148,0.035450,30.191312,1286.072148,31.069261,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,0,LL1_275-FI#3-R2_CRR_DiadFit,103.049046,0.017057,0.017057,1286.160574,0.000000,57.618504,1286.160574,64.479381,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## First, get the metadata for this day

In [5]:
# Put the common string you can in all your Ne lines here
ID_str='_'
file_ext_meta='txt' # Will only take files of this type 
diad_meta=pf.get_files(path=meta_path,
file_ext='txt', exclude_str=['N', 'Si', 'series','IMG','Cap','window','nodiad'],
 sort=True)
diad_meta

['LL1_275-FI#2-R2.txt',
 'LL1_275-FI#2-R3.txt',
 'LL1_275-FI#2-R4.txt',
 'LL1_275-FI#3-R1.txt',
 'LL1_275-FI#3-R2.txt',
 'LL4_2-FI#2.txt',
 'LL8_156-FI#1-6mw-R1.txt',
 'LL8_156-FI#1-6mw-R2.txt',
 'LL8_156-FI#1-6mw-R3.txt',
 'LL8_156-FI#2.txt',
 'LL8_156-FI#4-12mw.txt',
 'LL8_156-FI#4-6mw-R1.txt',
 'LL8_156-FI#4-6mw-R2.txt',
 'LL8_156-FI#4-6mw-R3.txt',
 'LL8_156-FI#6-12mw-R1.txt',
 'LL8_156-FI#6-12mw-R2.txt',
 'LL8_156-FI#6-12mw-R3.txt',
 'LL8_156-FI#6-6mw.txt',
 'LL8_243-FI#2-12mw-R1.txt',
 'LL8_243-FI#2-12mw-R2.txt',
 'LL8_243-FI#2-12mw-R3.txt',
 'LL8_406-FI#4-12mw.txt',
 'de (1).txt',
 'de (2).txt',
 'de (3).txt',
 'de (4).txt',
 'de (5).txt',
 'de (6).txt',
 'de (7).txt',
 'de.txt']

## Now get all the important information out of the metadata files

In [6]:
meta=pf.stitch_metadata_in_loop_witec(path=meta_path, 
Allfiles=diad_meta, prefix=False,
trupower=True)
meta['filename'].iloc[0]

100%|██████████| 30/30 [00:01<00:00, 17.69it/s]

Done


'LL4_2-FI#2.txt'

## Then get a simple file name you can stitch with the spectra

In [7]:
file_m=pf.extracting_filenames_generic(names=meta['filename'],
    prefix=False, str_prefix=" ",
   file_type='.txt')
# Checks they are all unique
file_m[0]

good job, no duplicate file names


'LL4_2-FI#2'

## Now get filename from spectra into same form as metadata

In [8]:
# Remove these to get the pure file name
file_s=pf.extracting_filenames_generic(
    prefix=False, str_prefix=" ",
    names=df2['filename'].reset_index(drop=True),
   file_type='.txt')

# Remove the cosmic rays, as doesnt have it in the metatdata
file_s = np.char.replace(file_s.astype(str), "_CRR_DiadFit", "")

good job, no duplicate file names


In [9]:
file_s

array(['LL1_275-FI#2-R2', 'LL1_275-FI#2-R3', 'LL1_275-FI#2-R4',
       'LL1_275-FI#3-R1', 'LL1_275-FI#3-R2', 'LL4_2-FI#2',
       'LL8_156-FI#1-6mw-R1', 'LL8_156-FI#1-6mw-R2',
       'LL8_156-FI#1-6mw-R3', 'LL8_156-FI#2', 'LL8_156-FI#4-6mw-R1',
       'LL8_156-FI#4-6mw-R2', 'LL8_156-FI#4-6mw-R3',
       'LL8_156-FI#6-12mw-R1', 'LL8_156-FI#6-12mw-R2',
       'LL8_156-FI#6-12mw-R3', 'LL8_156-FI#6-6mw', 'LL8_243-FI#2-12mw-R1',
       'LL8_243-FI#2-12mw-R2', 'LL8_243-FI#2-12mw-R3',
       'LL8_406-FI#4-12mw', 'LL8_156-FI#4-12mw'], dtype='<U20')

In [10]:
# Combining them together
meta['name_for_matching']=file_m
df2['name_for_matching']=file_s
df2['Name_for_Secondary_Phases']=file_s
df_combo=df2.merge(meta, on='name_for_matching')

In [11]:
Ne_corr=pf.calculate_Ne_corr_std_err_values(pickle_str='polyfit_data.pkl', 
    new_x=df_combo['sec since midnight'], CI=0.67)
Ne_corr.head()


1


,time,preferred_values,lower_values,upper_values
0,53484.0,0.997555,0.997543,0.997567
1,53637.0,0.997555,0.997543,0.997567
2,53858.0,0.997555,0.997542,0.997567
3,52813.0,0.997556,0.997543,0.997568
4,53064.0,0.997555,0.997543,0.997568


In [12]:

split_err=pf.propagate_errors_for_splitting(Ne_corr, df_combo)
df_combo_out=df_combo.copy()
df_combo_out.insert(1, 'Corrected_Splitting', df_combo['Splitting']*Ne_corr['preferred_values'])
df_combo_out.insert(2, 'Corr_Split+1σ', df_combo_out['Corrected_Splitting']+split_err)
df_combo_out.insert(3, 'Corr_Split-1σ',df_combo_out['Corrected_Splitting']-split_err)
df_combo_out.insert(4, 'Corr_Split_1σ_val', split_err)

nm=os.path.basename(DayFolder)
df_combo_out.to_excel(output_path+'/'+nm+'_FI_fitting.xlsx')

## If you have secondary phases, now is the time to merge those in

In [13]:
if path.exists('Carb_Peak_fits.xlsx'):
    Carb=pd.read_excel('Carb_Peak_fits.xlsx')
else:
    Carb=None
if path.exists('SO2_Peak_fits.xlsx'):
    SO2=pd.read_excel('SO2_Peak_fits.xlsx')
else:
    SO2=None
if SO2 is not None and Carb is not None:
    Sec_Phases=pd.merge(SO2, Carb, on='filename', how='outer')
elif SO2 is not None and Carb is None:
    Sec_Phases=SO2
elif SO2 is None and Carb is not None:
    Sec_Phases=Carb
else:
    Sec_Phases=None
Sec_Phases.head()

,filename,Peak_Cent_SO2,Peak_Area_SO2,Peak_Height_SO2,Model_name
0,LL4_2-FI#2.txt,1150.710585,63.339460,45.266549,Spline
1,LL8_156-FI#1-6mw-R1.txt,1150.622093,34.602552,16.777434,Spline
2,LL8_156-FI#1-6mw-R2.txt,1150.681087,28.364822,21.406792,Spline
3,LL8_156-FI#1-6mw-R3.txt,1150.710585,31.044180,23.323566,Spline
4,LL8_156-FI#2.txt,1150.769579,203.687957,132.549560,Spline


In [14]:
# Remove these to get the pure file name
if Sec_Phases is not None:
    file_sec_phase=pf.extracting_filenames_generic(
        prefix=False, str_prefix=" ",
        names=Sec_Phases['filename'].reset_index(drop=True),
       file_type='.txt')

    file_sec_phase

good job, no duplicate file names


In [15]:
file_sec_phase

array(['LL4_2-FI#2', 'LL8_156-FI#1-6mw-R1', 'LL8_156-FI#1-6mw-R2',
       'LL8_156-FI#1-6mw-R3', 'LL8_156-FI#2', 'LL8_156-FI#4-12mw',
       'LL8_156-FI#4-6mw-R1', 'LL8_156-FI#4-6mw-R2',
       'LL8_156-FI#4-6mw-R3', 'LL8_156-FI#6-12mw-R1',
       'LL8_156-FI#6-12mw-R2', 'LL8_156-FI#6-12mw-R3', 'LL8_156-FI#6-6mw',
       'LL8_243-FI#2-12mw-R1', 'LL8_243-FI#2-12mw-R2',
       'LL8_243-FI#2-12mw-R3'], dtype=object)

In [16]:
df_combo['name_for_matching']

0          LL1_275-FI#2-R2
1          LL1_275-FI#2-R3
2          LL1_275-FI#2-R4
3          LL1_275-FI#3-R1
4          LL1_275-FI#3-R2
5               LL4_2-FI#2
6      LL8_156-FI#1-6mw-R1
7      LL8_156-FI#1-6mw-R2
8      LL8_156-FI#1-6mw-R3
9             LL8_156-FI#2
10     LL8_156-FI#4-6mw-R1
11     LL8_156-FI#4-6mw-R2
12     LL8_156-FI#4-6mw-R3
13    LL8_156-FI#6-12mw-R1
14    LL8_156-FI#6-12mw-R2
15    LL8_156-FI#6-12mw-R3
16        LL8_156-FI#6-6mw
17    LL8_243-FI#2-12mw-R1
18    LL8_243-FI#2-12mw-R2
19    LL8_243-FI#2-12mw-R3
20       LL8_406-FI#4-12mw
21       LL8_156-FI#4-12mw
Name: name_for_matching, dtype: object

In [17]:
df_combo['name_for_matching']=df_combo['Name_for_Secondary_Phases']

if Sec_Phases is not None:
    Sec_Phases['name_for_matching']=file_sec_phase
    df_combo_sec_phase=df_combo_out.merge(Sec_Phases, 
    on='name_for_matching', how='outer')
    
else:
    df_combo_sec_phase=df_combo_out

### Add some more useful columns

In [18]:
if 'Peak_Area_Carb' in df_combo_sec_phase.columns:
    df_combo_sec_phase['Carb_Diad_Ratio']=(df_combo_sec_phase['Peak_Area_Carb']/(df_combo_sec_phase['Diad1_Voigt_Area']
                      +df_combo_sec_phase['Diad2_Voigt_Area']))
if 'Peak_Area_SO2' in df_combo_sec_phase.columns:
    df_combo_sec_phase['SO2_Diad_Ratio']=(df_combo_sec_phase['Peak_Area_SO2']/(df_combo_sec_phase['Diad1_Voigt_Area']
                      +df_combo_sec_phase['Diad2_Voigt_Area']))
    
df_combo_sec_phase.head()

,Unnamed: 0,Corrected_Splitting,Corr_Split+1σ,Corr_Split-1σ,Corr_Split_1σ_val,filename_x,Splitting,Split_err_abs,Split_err_quadrature,Diad1_Combofit_Cent,...,duration,24hr_time,sec since midnight,Spectral Center,filename,Peak_Cent_SO2,Peak_Area_SO2,Peak_Height_SO2,Model_name,SO2_Diad_Ratio
0,0,103.042338,103.138485,102.946191,0.096147,LL1_275-FI#2-R2_CRR_DiadFit,103.294890,0.133473,0.096374,1285.690490,...,"['0h', '2m', '26s']",2:51:24 PM\n,53484.0,1325.004,NaN,NaN,NaN,NaN,NaN,NaN
1,0,102.936073,102.962730,102.909415,0.026658,LL1_275-FI#2-R3_CRR_DiadFit,103.188372,0.037022,0.026692,1285.834584,...,"['0h', '3m', '1s']",2:53:57 PM\n,53637.0,1325.004,NaN,NaN,NaN,NaN,NaN,NaN
2,0,102.952580,102.962489,102.942672,0.009909,LL1_275-FI#2-R4_CRR_DiadFit,103.204924,0.009848,0.009848,1285.782739,...,"['0h', '3m', '1s']",2:57:38 PM\n,53858.0,1325.004,NaN,NaN,NaN,NaN,NaN,NaN
3,0,102.886464,102.928522,102.844405,0.042059,LL1_275-FI#3-R1_CRR_DiadFit,103.138565,0.058238,0.042143,1286.072148,...,"['0h', '3m', '47s']",2:40:13 PM\n,52813.0,1325.004,NaN,NaN,NaN,NaN,NaN,NaN
4,0,102.797130,102.814193,102.780067,0.017063,LL1_275-FI#3-R2_CRR_DiadFit,103.049046,0.017057,0.017057,1286.160574,...,"['0h', '3m', '2s']",2:44:24 PM\n,53064.0,1325.004,NaN,NaN,NaN,NaN,NaN,NaN


In [19]:
df_combo_sec_phase.to_excel(output_path+'/'+nm+'_FI_fitting_w_sec_phases.xlsx')

In [20]:
from winotify import Notification, audio

toast= Notification(app_id="VSCode",title="Notebook completed",
                    msg="Step4_Stitch_Outputs_Together is done!",
                    duration="short")
toast.set_audio(audio.Mail,loop=False)
toast.show()